In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content/drive/Shareddrives/CSE573/preprocessed


In [ ]:
%cd /content/drive/Shareddrives/CSE573/preprocessed
%ls

/content/drive/Shareddrives/CSE573/preprocessed
AaplAmzn_NewsDateDirection.csv
Aapl_NewsPriceHistDirection.csv
Aapl_SentimentAggMeanPriceHistDirection.csv
Aapl_SentimentAggSumPriceHistDirection.csv
Amzn_NewsPriceHistDirection.csv
Amzn_SentimentAggMeanPriceHistDirection.csv
Amzn_SentimentAggSumPriceHistDirection.csv
amzn_title_processedText_direction.csv
amzn_title_processedText_direction_date.csv
amzn_title_text_label.csv
lstm_amzn_fulltext/


In [ ]:
#!pip install keras
#!pip install matplotlib-venn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

import requests
import io

In [ ]:
import random

filename = 'Aapl_NewsPriceHistDirection.csv'
n = 69622 #number of records in file (excludes header)
print(n)
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df_aapl = pd.read_csv(filename, skiprows=skip)
df_aapl

69622


,uuid,author,url,title,language,text,published,title_aapl,text_aapl,title_amzn,...,prevClose_16,prevClose_17,prevClose_18,prevClose_19,prevClose_20,prevClose_21,prevClose_22,prevClose_23,prevClose_24,prevClose_25
0,3f884eef1ee888c2053c8eed3dda913e54a568f9,admin,http://www.wallstreetreporter.com/2018/09/the-...,The evidence is mounting that Apple&apos;s nex...,english,"September 2, 2018 admin Trading Ideas Comments...",2018-09-02 12:00:00+00:00,True,True,False,...,207.51,208.78,206.92,207.09,208.99,207.98,207.42,201.64,195.64,189.98
1,7a0a866f1f3a387d62f18695fd450c86d2de8616,morningstar.com,http://news.morningstar.com/all/dow-jones/mark...,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,english,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",2018-09-04 00:00:00+00:00,False,True,False,...,207.51,208.78,206.92,207.09,208.99,207.98,207.42,201.64,195.64,189.98
2,dc375f6a0d63423a29cb086ac3cfd2470ce6ba74,John Vincent,https://seekingalpha.com/article/4203642-track...,Tracking David Einhorn's Portfolio - Q2 2018 U...,english,This article is part of a series that provides...,2018-09-02 17:34:00+00:00,False,True,False,...,207.51,208.78,206.92,207.09,208.99,207.98,207.42,201.64,195.64,189.98
3,3cc184fb4377bd97fedc61572edc38f9680ec764,Andy Crowder,http://www.wyattresearch.com/article/multiplyi...,Multiplying my 247% Profit in Pot Stocks,english,"Frankly, I’ve just developed a brand-new tradi...",2018-09-02 13:30:00+00:00,False,True,False,...,207.51,208.78,206.92,207.09,208.99,207.98,207.42,201.64,195.64,189.98
4,7bd4df56251c565f32426a48eacde2d450248d44,morningstar.com,http://news.morningstar.com/all/benzinga/BenzB...,Mike Khouw's Apple Options Trade,english,"On CNBC's ""Options Action"" , Mike Khouw recomm...",2018-09-04 00:00:00+00:00,True,True,False,...,207.51,208.78,206.92,207.09,208.99,207.98,207.42,201.64,195.64,189.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0fb4c6add332a928f4431a51665dfffc33c58885,Wealthy Content,https://wealthycontent.com/2018/06/07/advantag...,Advantages of the Individual Investor - Wealth...,english,Advantages of the Individual Investor Posted o...,2018-06-07 00:00:00+00:00,False,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
9996,1550edadbae03c95ad9225dca852d0fdc65d51e5,James Anthony,http://assetsstock.com/business/global-seed-pr...,Global Seed Processing Machine Market 2018 Per...,english,Search for: Global Seed Processing Machine Mar...,2018-06-07 13:00:00+00:00,False,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
9997,4a20d4d076e364d95329c3797f2e0540528f7cd7,NaN,http://www.blackamericans.com/business-news/58...,Dark mode looks great on the Mac — but it's iO...,english,advertisement Business Dark mode looks great o...,2018-06-07 01:55:00+00:00,True,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36
9998,f450e9cb670f0332693fe533215539ce3fac4db4,Logan Wallace,https://www.tickerreport.com/banking-finance/3...,UBS Group Analysts Give Apple (AAPL) a $210.00...,english,Tweet\nApple (NASDAQ:AAPL) received a $210.00 ...,2018-06-07 11:06:00+00:00,True,True,False,...,186.23,188.00,188.57,189.51,187.36,185.89,185.00,183.81,176.92,176.36


In [ ]:
df_aapl = df_aapl[["title", "text", "direction", "prediction_date"]]

In [ ]:
df_aapl

,title,text,direction,prediction_date
0,The evidence is mounting that Apple&apos;s nex...,"September 2, 2018 admin Trading Ideas Comments...",1,2018-09-04
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"By Mark DeCambre, MarketWatch , Ryan Vlastelic...",1,2018-09-04
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,This article is part of a series that provides...,1,2018-09-04
3,Multiplying my 247% Profit in Pot Stocks,"Frankly, I’ve just developed a brand-new tradi...",1,2018-09-04
4,Mike Khouw's Apple Options Trade,"On CNBC's ""Options Action"" , Mike Khouw recomm...",1,2018-09-04
...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,Advantages of the Individual Investor Posted o...,-1,2018-06-07
9996,Global Seed Processing Machine Market 2018 Per...,Search for: Global Seed Processing Machine Mar...,-1,2018-06-07
9997,Dark mode looks great on the Mac — but it's iO...,advertisement Business Dark mode looks great o...,-1,2018-06-07
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,Tweet\nApple (NASDAQ:AAPL) received a $210.00 ...,-1,2018-06-07


In [ ]:
df_aapl['text'] = df_aapl['text'].apply(lambda x: x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_aapl

,title,text,direction,prediction_date
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04
...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07


In [ ]:
import string

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#stopwords.words('english')
df_aapl['text_without_stopwords'] = df_aapl['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

df_aapl['text_without_stopwords'] = df_aapl['text_without_stopwords'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in string.punctuation]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df_aapl

,title,text,direction,prediction_date,text_without_stopwords
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04,september 2 2018 admin trading ideas comments ...
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04,mark decambre marketwatch ryan vlastelica s p ...
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04,article part series provides ongoing analysis ...
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04,frankly i ’ ve developed brand-new trading str...
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04,cnbc 's `` options action '' mike khouw recomm...
...,...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07,advantages individual investor posted wealthy ...
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07,search for global seed processing machine mark...
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07,advertisement business dark mode looks great m...
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07,tweet apple nasdaq aapl received 210.00 price ...


In [ ]:
common_words=pd.Series(''.join(df_aapl['text_without_stopwords']).split()).value_counts()
common_words

shares          71775
apple           69211
’               55828
s               55425
stock           47734
                ...  
unaffordable        1
85.19m              1
-24.66              1
108.01              1
-11.63              1
Length: 193021, dtype: int64

In [ ]:
common_words = common_words.drop(labels=[x for x in common_words.index if common_words[x] < 100])

In [ ]:
common_words

shares           71775
apple            69211
’                55828
s                55425
stock            47734
                 ...  
critics            100
comeback           100
nav                100
restructuring      100
impacts            100
Length: 5629, dtype: int64

In [ ]:
common_words = common_words.drop(labels=['“'])

In [ ]:
common_words

shares           71775
apple            69211
stock            47734
company          42573
rating           32692
                 ...  
critics            100
comeback           100
nav                100
restructuring      100
impacts            100
Length: 5624, dtype: int64

In [ ]:

texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if word in common_words:
            final_words.append(word)
    return ' '.join(final_words)

df_aapl['processedText'] = df_aapl['text_without_stopwords'].apply(lambda x: removeWords(word_tokenize(x)))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df_aapl

,title,text,direction,prediction_date,text_without_stopwords,processedText
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04,september 2 2018 admin trading ideas comments ...,september 2 2018 trading ideas comments eviden...
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04,mark decambre marketwatch ryan vlastelica s p ...,mark decambre marketwatch ryan vlastelica p 50...
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04,article part series provides ongoing analysis ...,article part series provides ongoing analysis ...
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04,frankly i ’ ve developed brand-new trading str...,i ve developed trading allowing us profit amer...
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04,cnbc 's `` options action '' mike khouw recomm...,cnbc `` options action `` mike recommended lon...
...,...,...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07,advantages individual investor posted wealthy ...,advantages individual investor posted content ...
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07,search for global seed processing machine mark...,search for global processing machine market 20...
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07,advertisement business dark mode looks great m...,advertisement business mode looks great mac — ...
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07,tweet apple nasdaq aapl received 210.00 price ...,tweet apple nasdaq aapl received 210.00 price ...


In [ ]:
df_aapl.to_csv('aapl_text_direction_date.csv', index=False)

In [ ]:
df_temp = pd.read_csv('aapl_text_direction_date.csv')
df_temp

,title,text,direction,prediction_date,text_without_stopwords,processedText
0,The evidence is mounting that Apple&apos;s nex...,"september 2, 2018 admin trading ideas comments...",1,2018-09-04,september 2 2018 admin trading ideas comments ...,september 2 2018 trading ideas comments eviden...
1,MARKET SNAPSHOT: Stocks Sag With Nafta In Focu...,"by mark decambre, marketwatch , ryan vlastelic...",1,2018-09-04,mark decambre marketwatch ryan vlastelica s p ...,mark decambre marketwatch ryan vlastelica p 50...
2,Tracking David Einhorn's Portfolio - Q2 2018 U...,this article is part of a series that provides...,1,2018-09-04,article part series provides ongoing analysis ...,article part series provides ongoing analysis ...
3,Multiplying my 247% Profit in Pot Stocks,"frankly, i’ve just developed a brand-new tradi...",1,2018-09-04,frankly i ’ ve developed brand-new trading str...,i ve developed trading allowing us profit amer...
4,Mike Khouw's Apple Options Trade,"on cnbc's ""options action"" , mike khouw recomm...",1,2018-09-04,cnbc 's `` options action '' mike khouw recomm...,cnbc `` options action `` mike recommended lon...
...,...,...,...,...,...,...
9995,Advantages of the Individual Investor - Wealth...,advantages of the individual investor posted o...,-1,2018-06-07,advantages individual investor posted wealthy ...,advantages individual investor posted content ...
9996,Global Seed Processing Machine Market 2018 Per...,search for: global seed processing machine mar...,-1,2018-06-07,search for global seed processing machine mark...,search for global processing machine market 20...
9997,Dark mode looks great on the Mac — but it's iO...,advertisement business dark mode looks great o...,-1,2018-06-07,advertisement business dark mode looks great m...,advertisement business mode looks great mac — ...
9998,UBS Group Analysts Give Apple (AAPL) a $210.00...,tweet\napple (nasdaq:aapl) received a $210.00 ...,-1,2018-06-07,tweet apple nasdaq aapl received 210.00 price ...,tweet apple nasdaq aapl received 210.00 price ...
